# 第2章: JuMPによる線形計画法の基本 (Julia版)

この章では、JuMPを使って線形計画問題を解く基本的な方法を学びます。

In [1]:
using JuMP
using HiGHS

## 2.1 JuMPの基本

JuMPは数学的な記法に近い形で最適化問題を定義できます。

### 基本的な流れ

1. `Model()` でモデルを作成
2. `@variable` で決定変数を定義
3. `@constraint` で制約条件を追加
4. `@objective` で目的関数を設定
5. `optimize!()` で最適化を実行
6. `value()` で結果を取得

### 主要なマクロ

| マクロ | 説明 |
|--------|------|
| `@variable` | 決定変数の定義 |
| `@constraint` | 制約条件の追加 |
| `@objective` | 目的関数の設定 |
| `@expression` | 式の定義（再利用可能） |

## 例題1: 基本的な最大化問題

### 問題

**最大化:** $z = x + 2y$

**制約条件:**
- $2x + y \leq 20$
- $-4x + 5y \leq 10$
- $-x + 2y \geq -2$
- $x \geq 0, y \geq 0$

In [2]:
# 1. モデルの作成
model = Model(HiGHS.Optimizer)
set_silent(model)  # ソルバーの出力を抑制

# 2. 決定変数の定義
@variable(model, x >= 0)
@variable(model, y >= 0)

# 3. 制約条件の追加
@constraint(model, red_constraint, 2x + y <= 20)
@constraint(model, blue_constraint, -4x + 5y <= 10)
@constraint(model, yellow_constraint, -x + 2y >= -2)

# 4. 目的関数の設定
@objective(model, Max, x + 2y)

# 5. 最適化実行
optimize!(model)

# 6. 結果表示
println("【結果】")
println("ステータス: ", termination_status(model))
println("最大値: ", round(objective_value(model), digits=4))
println("x = ", round(value(x), digits=4))
println("y = ", round(value(y), digits=4))

println("\n【解釈】")
println("最適解は x≈6.43, y≈7.14 で、最大値は約20.71です。")

【結果】


ステータス: OPTIMAL
最大値: 

20.7143
x = 6.4286
y = 7.1429

【解釈】
最適解は x≈6.43, y≈7.14 で、最大値は約20.71です。


## 例題2: 基本的な最小化問題

### 問題

**最小化:** $z = 3x + 2y$

**制約条件:**
- $x + y \geq 4$
- $2x + y \geq 6$
- $x \geq 0, y \geq 0$

In [3]:
model = Model(HiGHS.Optimizer)
set_silent(model)

@variable(model, x >= 0)
@variable(model, y >= 0)

@constraint(model, x + y >= 4)
@constraint(model, 2x + y >= 6)

@objective(model, Min, 3x + 2y)

optimize!(model)

println("【結果】")
println("ステータス: ", termination_status(model))
println("最小値: ", round(objective_value(model), digits=4))
println("x = ", round(value(x), digits=4))
println("y = ", round(value(y), digits=4))

println("\n【解釈】")
println("最小値は x=2, y=2 のとき z=10 です。")

【結果】


ステータス: OPTIMAL
最小値: 10.0
x = 2.0
y = 2.0

【解釈】
最小値は x=2, y=2 のとき z=10 です。


## 例題3: 等式制約を含む問題

### 問題

**最大化:** $z = x + 2y$

**制約条件:**
- $2x + y \leq 20$
- $-4x + 5y \leq 10$
- $-x + 2y \geq -2$
- $-x + 5y = 15$（等式制約）
- $x \geq 0, y \geq 0$

In [4]:
model = Model(HiGHS.Optimizer)
set_silent(model)

@variable(model, x >= 0)
@variable(model, y >= 0)

# 不等式制約
@constraint(model, 2x + y <= 20)
@constraint(model, -4x + 5y <= 10)
@constraint(model, -x + 2y >= -2)

# 等式制約
@constraint(model, -x + 5y == 15)

@objective(model, Max, x + 2y)

optimize!(model)

println("【結果】")
println("ステータス: ", termination_status(model))
println("最大値: ", round(objective_value(model), digits=4))
println("x = ", round(value(x), digits=4))
println("y = ", round(value(y), digits=4))

【結果】
ステータス: OPTIMAL
最大値: 16.8182
x = 7.7273
y = 4.5455


## 例題4: リソース配分問題（工場の生産計画）

### 問題

ある工場で4種類の製品を生産しています。各製品の利益と資源消費量は以下の通りです：

|  | 利益 | 人員 | 原材料A | 原材料B |
|--|------|------|---------|--------|
| 製品1 | \$20 | 1 | 3 | 0 |
| 製品2 | \$12 | 1 | 2 | 1 |
| 製品3 | \$40 | 1 | 1 | 2 |
| 製品4 | \$25 | 1 | 0 | 3 |

**制約：**
- 1日の総生産量は50個以下（人員制約）
- 原材料Aは100単位まで使用可能
- 原材料Bは90単位まで使用可能

**目標:** 利益を最大化する生産量を求める

In [5]:
model = Model(HiGHS.Optimizer)
set_silent(model)

# データ定義
products = 1:4
profit = [20, 12, 40, 25]
manpower = [1, 1, 1, 1]
material_a = [3, 2, 1, 0]
material_b = [0, 1, 2, 3]

# 決定変数
@variable(model, x[products] >= 0)

# 制約条件
@constraint(model, 人員制約, sum(manpower[i] * x[i] for i in products) <= 50)
@constraint(model, 原材料A, sum(material_a[i] * x[i] for i in products) <= 100)
@constraint(model, 原材料B, sum(material_b[i] * x[i] for i in products) <= 90)

# 目的関数
@objective(model, Max, sum(profit[i] * x[i] for i in products))

optimize!(model)

println("【結果】")
println("ステータス: ", termination_status(model))
println("最大利益: \$", round(objective_value(model), digits=2))

println("\n生産計画:")
for i in products
    println("  製品$i: ", round(value(x[i]), digits=2), " 個")
end

println("\n【解釈】")
println("最適解は製品1を5個、製品3を45個生産することです。")
println("製品2と製品4は利益率が低いため生産しません。")

【結果】


ステータス: OPTIMAL
最大利益: $1900.0

生産計画:
  製品1: 5.0 個
  製品2: 0.0 個
  製品3: 45.0 個
  製品4: 0.0 個

【解釈】
最適解は製品1を5個、製品3を45個生産することです。
製品2と製品4は利益率が低いため生産しません。


## 例題5: 変数の範囲制約

### 問題

**最小化:** $z = 2x + 3y$

**制約条件:**
- $x + y \leq 10$
- $2 \leq x \leq 8$（xの範囲）
- $1 \leq y \leq 6$（yの範囲）

In [6]:
model = Model(HiGHS.Optimizer)
set_silent(model)

# 変数の範囲を指定
@variable(model, 2 <= x <= 8)
@variable(model, 1 <= y <= 6)

@constraint(model, x + y <= 10)

@objective(model, Min, 2x + 3y)

optimize!(model)

println("【結果】")
println("ステータス: ", termination_status(model))
println("最小値: ", round(objective_value(model), digits=4))
println("x = ", round(value(x), digits=4))
println("y = ", round(value(y), digits=4))

println("\n【解釈】")
println("最小値は x=2, y=1 のとき z=7 です。")

【結果】
ステータス: OPTIMAL
最小値: 7.0
x = 2.0
y = 1.0

【解釈】
最小値は x=2, y=1 のとき z=7 です。


## 例題6: 輸送問題（簡易版）

### 問題

2つの工場（A, B）から2つの店舗（1, 2）へ商品を輸送します。

**輸送コスト（\$/個）:**

|  | 店舗1 | 店舗2 |
|--|-------|-------|
| 工場A | 4 | 6 |
| 工場B | 5 | 3 |

- 供給量: 工場A=50, 工場B=40
- 需要量: 店舗1=30, 店舗2=60

**目標:** 総輸送コストを最小化

In [7]:
model = Model(HiGHS.Optimizer)
set_silent(model)

# データ
factories = [:A, :B]
stores = [1, 2]
supply = Dict(:A => 50, :B => 40)
demand = Dict(1 => 30, 2 => 60)
cost = Dict(
    (:A, 1) => 4, (:A, 2) => 6,
    (:B, 1) => 5, (:B, 2) => 3
)

# 決定変数
@variable(model, x[f in factories, s in stores] >= 0)

# 供給制約
for f in factories
    @constraint(model, sum(x[f, s] for s in stores) == supply[f])
end

# 需要制約
for s in stores
    @constraint(model, sum(x[f, s] for f in factories) == demand[s])
end

# 目的関数
@objective(model, Min, sum(cost[f, s] * x[f, s] for f in factories, s in stores))

optimize!(model)

println("【結果】")
println("ステータス: ", termination_status(model))
println("最小輸送コスト: \$", round(objective_value(model), digits=2))

println("\n輸送計画:")
for f in factories, s in stores
    v = value(x[f, s])
    if v > 0
        println("  工場$f → 店舗$s: ", round(v, digits=0), " 個")
    end
end

【結果】


ステータス: OPTIMAL
最小輸送コスト: $360.0

輸送計画:
  工場A → 店舗1: 30.0 個
  工場A → 店舗2: 20.0 個
  工場B → 店舗2: 40.0 個


## 第2章のまとめ

この章で学んだこと：

1. **JuMPの基本的な使い方**
   - `Model()`: モデルの作成
   - `@variable`: 決定変数の定義
   - `@constraint`: 制約条件の追加
   - `@objective`: 目的関数の設定

2. **最大化・最小化の指定**
   - `Max` または `Min` を使用

3. **制約条件の種類**
   - `<=`, `>=`, `==` をそのまま使用可能

4. **変数の範囲制約**
   - `@variable(model, a <= x <= b)` 形式で指定

5. **配列・辞書を使った定義**
   - 複数変数を効率的に管理
   - `sum()` で合計を簡潔に表現

---

**次の章:** より多くの例題でJuMPの使い方を深めます！